In [198]:
!pip install --upgrade pip
!pip install pandas==1.5.3 seaborn==0.12.2 matplotlib==3.7.1

In [199]:
# Importamos las librerias necesarias.
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re

In [200]:
# Cargamos la data en la variable df.
df = pd.read_csv('/content/incidentes_viales.csv', delimiter=';')

<ipython-input-200-553e48cecddb>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/incidentes_viales.csv', delimiter=';')


In [201]:
print("La cantidad de filas que tenemos es:", df.shape[0],
      "\nY la cantidad de columnas es:", df.shape[1],
      "\n\nLas primeras columnas de nuestro data set nos arrojan la siguiente información:")

df.head()

La cantidad de filas que tenemos es: 270765 
Y la cantidad de columnas es: 18 

Las primeras columnas de nuestro data set nos arrojan la siguiente información:


,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DIRECCION ENCASILLADA,DISEÑO,EXPEDIENTE,FECHA_ACCIDENTE,FECHA_ACCIDENTES,GRAVEDAD_ACCIDENTE,MES,NRO_RADICADO,NUMCOMUNA,BARRIO,COMUNA,LOCATION,X,Y
0,2015,1013,Caida Ocupante,CR 46 CL 43,CR 046 043 000 00000,Tramo de via,A000259731,21/10/2015 05:58:00,2015-10-21T10:58:00.000Z,Con heridos,10,1508668,10,Barrio Colón,La Candelaria,"[-75.5688011014, 6.24312304123]",834949.69,1182357.34
1,2015,1020,Choque,CR 43 A CL 29,CR 043 A 029 000 00000,Tramo de via,A000261725,05/11/2015 03:00:00,2015-11-05T08:00:00.000Z,Solo daños,11,1510621,10,San Diego,La Candelaria,"[-75.5693883283, 6.22870030622]",834880.17,1180762.02
2,2015,0205,Otro,CR 46 CL 37,CR 046 037 000 00000,Tramo de via,A000259739,21/10/2015 07:40:00,2015-10-21T12:40:00.000Z,Con heridos,10,1508691,02,La Francia,Santa Cruz,"[-75.5503911403, 6.29869502502]",837004.94,1188499.17
3,2015,0101,Atropello,CR 34 CL 112,CR 034 112 000 00000,Tramo de via,A000259752,21/10/2015 12:00:00,2015-10-21T17:00:00.000Z,Con heridos,10,1508694,01,Santo Domingo Savio No. 1,Popular,"[-75.5442195069, 6.30019383306]",837688.53,1188663.05
4,2015,1504,Choque,CR 50 CL 14,CR 050 014 000 00000,Tramo de via,A000259949,03/11/2015 06:16:00,2015-11-03T11:16:00.000Z,Solo daños,11,1510634,15,Santa Fé,Guayabal,"[-75.578180998, 6.21699243701]",833903.11,1179469.60


In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270765 entries, 0 to 270764
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   AÑO                    270765 non-null  object 
 1   CBML                   252609 non-null  object 
 2   CLASE_ACCIDENTE        270759 non-null  object 
 3   DIRECCION              270765 non-null  object 
 4   DIRECCION ENCASILLADA  270374 non-null  object 
 5   DISEÑO                 269617 non-null  object 
 6   EXPEDIENTE             270655 non-null  object 
 7   FECHA_ACCIDENTE        270765 non-null  object 
 8   FECHA_ACCIDENTES       270765 non-null  object 
 9   GRAVEDAD_ACCIDENTE     270765 non-null  object 
 10  MES                    270765 non-null  int64  
 11  NRO_RADICADO           270760 non-null  object 
 12  NUMCOMUNA              270765 non-null  object 
 13  BARRIO                 251759 non-null  object 
 14  COMUNA                 257967 non-nu

Con lo anterior podemos darnos cuenta que tenemos varios datos faltantes en las siguientes columnas:

- CBML = Codigo catastral que corresponde al codigo comuna, barrio, manzana, lote catastral de un predio.
- CLASE_ACCIDENTE = Clasificación del IPAT sobre la clase de accidente de transito: choque, atropello, volcamiento, caida de ocupante, incendio, u otro (que no corresponde a las anteriores 5 clasificaciones, p. ej: sumersión)
- DIRECCION ENCASILLADA = Dirección encasillada que entrega el geocodificador
- DISEÑO = Sitio de la vía donde ocurrió el accidente: Cicloruta, Glorieta, Interseccion, Lote o Predio, Paso a Nivel, Paso Elevado, Paso Inferior, Pontón, Puente, Tramo de via, Tunel, Via peatonal
- EXPEDIENTE = Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
- NRO_RADICADO = Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
- BARRIO = Barrio de ocurrencia del incidente vial
- COMUNA = Denominación con la cual se identifica cada Comuna o Corregimiento. 01:Popular 02:Santa Cruz 03:Manrique 04:Aranjuez 05:Castilla 06:Doce de Octubre 07:Robledo 08:Villa Hermosa 09:Buenos Aires 10:La Candelaria 11:Laureles - Estadio 12:La América 13:San Javier 14:El Poblado 15:Guayabal 16:Belén 50:San Sebastián de Palmitas 60:San Cristobal 70:Altavista 80:San Antonio de Prado 90:Santa Elena 99:Toda la Ciudad

In [203]:
df.dtypes

AÑO                       object
CBML                      object
CLASE_ACCIDENTE           object
DIRECCION                 object
DIRECCION ENCASILLADA     object
DISEÑO                    object
EXPEDIENTE                object
FECHA_ACCIDENTE           object
FECHA_ACCIDENTES          object
GRAVEDAD_ACCIDENTE        object
MES                        int64
NRO_RADICADO              object
NUMCOMUNA                 object
BARRIO                    object
COMUNA                    object
LOCATION                  object
X                        float64
Y                        float64
dtype: object

In [204]:
# Validación de valores nulos
null = df.isnull().sum()
nullTable = pd.DataFrame({'Columna': df.columns, 'Valores nulos': null})
nullTable = nullTable.reset_index(drop=True)
nullTable

,Columna,Valores nulos
0,AÑO,0
1,CBML,18156
2,CLASE_ACCIDENTE,6
3,DIRECCION,0
4,DIRECCION ENCASILLADA,391
5,DISEÑO,1148
6,EXPEDIENTE,110
7,FECHA_ACCIDENTE,0
8,FECHA_ACCIDENTES,0
9,GRAVEDAD_ACCIDENTE,0


Cómo tenemos varios valores nulos, vamos a sacarlos del data set con la siguiente celda.

In [205]:
df = df[df['CBML'].isnull() != True]
df = df[df['CLASE_ACCIDENTE'].isnull() != True]
df = df[df['DIRECCION ENCASILLADA'].isnull() != True]
df = df[df['DISEÑO'].isnull() != True]
df = df[df['EXPEDIENTE'].isnull() != True]
df = df[df['NRO_RADICADO'].isnull() != True]
df = df[df['BARRIO'].isnull() != True]
df = df[df['COMUNA'].isnull() != True]

Ahora que no tenemos nulos, vamos a validar los valores duplicados que podemos tener.

In [206]:
# Validación de duplicados
duplicates = df.duplicated().sum()
duplicatesTable = pd.DataFrame({'Columna': df.columns, 'Duplicados': duplicates})
duplicatesTable

,Columna,Duplicados
0,AÑO,0
1,CBML,0
2,CLASE_ACCIDENTE,0
3,DIRECCION,0
4,DIRECCION ENCASILLADA,0
5,DISEÑO,0
6,EXPEDIENTE,0
7,FECHA_ACCIDENTE,0
8,FECHA_ACCIDENTES,0
9,GRAVEDAD_ACCIDENTE,0


Ya que no tenemos duplicados, vamos a empezar a validar los valores de las columnas que tenemos.

## Año

In [207]:
df['AÑO'].unique()

array(['2015', '2014', '2016', '2017', '2018', '2019\\r', '2019', '2020',
       2020, 2015, 2014, 2016, 2017, 2018], dtype=object)

Como vemos tenemos diversos valores para el mismo año, con ello en mente vamos a convertir la columna "AÑO" a tipo de datos string y luego vamos a extraer los primeros cuatro caracteres (que representan el año).

In [208]:
df["AÑO"] = df["AÑO"].astype(str).str[:4]
df["AÑO"].unique()

array(['2015', '2014', '2016', '2017', '2018', '2019', '2020'],
      dtype=object)

## CBML

In [209]:
df["CBML"].unique()

array(['1013', '1020', '0205', '0101', '1504', '1417', '1105', '1009',
       '1018', '0705', '0605', '1012', '0410', '0908', '0512', '0516',
       '0911', '1116', '0905', '8000', '1419', '1415', '0405', '1502',
       '0701', '1114', '0517', '1307', '0508', '0210', '0809', '1019',
       '1113', '1102', '1612', '1010', '0815', '0712', '1017', '0511',
       '0301', '1204', '0608', '0707', '1422', '0202', '1005', '1511',
       '0103', '0207', '0718', '1423', '1004', '0802', '1006', '1110',
       '1003', '0913', '0914', '0912', '1509', '0105', '1608', '0801',
       '0709', '1507', '1103', '0402', '0505', '1603', '0415', '1014',
       '1421', '1414', '1510', '0509', '0303', '0602', '0305', '1602',
       '0606', '0401', '0607', '1610', '1101', '0209', '7001', '1309',
       '1107', '1001', '1016', '1310', '0514', '0507', '1506', '1111',
       '1615', '0417', '0804', '1503', '1404', '1108', '0412', '0413',
       '0717', '0907', '0703', '1015', '6098', '0601', '1211', '1008',
      

En este caso para CBML no se denota un patron que podamos utilizar para eliminar datos que no nos aporte, así que no le realizaremos ninguna manipulación.

## CLASE_ACCIDENTE

In [210]:
df["CLASE_ACCIDENTE"].unique()

array(['Caida Ocupante', 'Choque', 'Otro', 'Atropello', 'Volcamiento',
       'Caída de Ocupante', 'Incendio'], dtype=object)

Cómo podemos ver tenemos registros con "Caída de Ocupante", vamos a reemplazar esto con "Caida de Ocupante" y vamos a pasar todas las opciones a minuscula.

In [211]:
df["CLASE_ACCIDENTE"] = df["CLASE_ACCIDENTE"].map(lambda x : str(x).replace("Caída de Ocupante", "Caida Ocupante").lower())
df["CLASE_ACCIDENTE"] = df["CLASE_ACCIDENTE"].str.capitalize()
df["CLASE_ACCIDENTE"].unique()

array(['Caida ocupante', 'Choque', 'Otro', 'Atropello', 'Volcamiento',
       'Incendio'], dtype=object)

## DIRECCION

Validadremos un ejemplo de 20 registros tomados al azar para validar las direcciones.

In [212]:
df["DIRECCION"].sample(20)

51757        CR 86 CL 49 E
191086         CR 69 CL 97
209638         CR 53 CL 49
154789         CL 57 CR 43
1576         CR 64 C CL 72
47074          CR 70 CL 51
112698    CR 64 C CL 111 D
270181      CL 111 CR 64 C
118214          CR 51 CL 5
17645          CR 65 CL 92
57814          CR 63 CL 50
103822       CR 80 CL 30 A
176843       CR 48 A CL 87
242960       CR 50 B CL 78
123904         CR 57 CL 57
87162        CL 47 D CR 86
129715         CR 57 CL 51
64742        CR 25 CL 10 E
61079        Tv 39 B CL 78
165780       CR 64 C CL 78
Name: DIRECCION, dtype: object

Por lo que podemos ver ya se encuentra correctamente esta columna, así que no será manipulada.

## DIRECCION ENCASILLADA

Validadremos un ejemplo de 20 registros tomados al azar para validar las direcciones encasilladas.

In [213]:
df["DIRECCION ENCASILLADA"].sample(20)

180625        CL  999   999  000 00000
127100      CL  078 B   071  000 00000
136923        CR  070   019  000 00000
98259         CR  050   025  000 00000
36923         CR  064   078  000 00000
17944       CL  063 A   040  000 00000
262027        CR  063   052  000 00000
247841        CL  065   078  000 00000
150597        CR  075   095  000 00000
268674        CL  120   063  000 00000
78172      CR  075 DA S 001  000 00000
173708    CL  047 F   085 A  000 00000
233803      CL  047 D   087  000 00000
110192        CR  063   034  000 00000
246397      CR  080 A   032  000 00000
26527         CL  068   033  000 00000
22845         CL  090   067  000 00000
50759         CR  073   073  000 00000
90232         CR  062   052  000 00000
48268     CR  064 E   103 D  000 00000
Name: DIRECCION ENCASILLADA, dtype: object

Por lo que podemos ver igual que en la columna anterior esta ya se encuentra correctamente, así que tampoco será manipulada.

## DISEÑO

In [214]:
df["DISEÑO"].unique()

array(['Tramo de via', 'Interseccion', 'Glorieta', 'Lote o Predio',
       'Paso a Nivel', 'Puente', 'Paso Elevado', 'Ciclo Ruta',
       'Paso Inferior', 'Via peatonal', 'Pontón', 'Tunel', 'Pont\\xF3n'],
      dtype=object)

Cómo podemos ver tenemos registros con "Pontón", vamos a reemplazar esto con "Ponton" y vamos a pasar todas las opciones a minuscula.

In [215]:
df["DISEÑO"] = df["DISEÑO"].map(lambda x : str(x).replace("Pont\\xF3n", "Ponton").lower())
df["DISEÑO"] = df["DISEÑO"].str.capitalize()
df["DISEÑO"].unique()

array(['Tramo de via', 'Interseccion', 'Glorieta', 'Lote o predio',
       'Paso a nivel', 'Puente', 'Paso elevado', 'Ciclo ruta',
       'Paso inferior', 'Via peatonal', 'Pontón', 'Tunel', 'Ponton'],
      dtype=object)

## EXPEDIENTE

In [216]:
df["EXPEDIENTE"].sample(20)

197303    A001011220
69795     A001162940
23213     A000773306
215151    A000920716
100344    A000922434
26801     A000841384
219061    A000665564
101518    A000855153
53031     A000782383
52949     A000781296
69962     A001163578
93090     A001153407
125679    A001074312
46135     A000186972
227507    A000667787
194860    A001003562
91635     A001148817
42534     A001077111
182357    A001150301
160461    A000490704
Name: EXPEDIENTE, dtype: object

En este caso para EXPEDIENTE no se denota un patron que podamos utilizar para eliminar datos que no nos aporte de forma similar a CBML, así que no le realizaremos ninguna manipulación.

## FECHA_ACCIDENTE y FECHA_ACCIDENTES

In [217]:
df["FECHA_ACCIDENTE"].sample(20)

158210    15/12/2014 10:30:00
236268    24/12/2015 09:10:00
56609     31/07/2014 14:10:00
32400     27/04/2019 17:20:00
106367    23/12/2017 17:30:00
208594    21/09/2018 12:45:00
42436     12/12/2019 23:40:00
201261    29/11/2019 14:30:00
11323     23/08/2016 08:30:00
238308    08/02/2016 06:20:00
123436    09/10/2019 07:30:00
249514    11/06/2016 13:50:00
137570    23/07/2014 06:40:00
123882    17/10/2019 10:30:00
211690    13/10/2018 16:40:00
67939     30/05/2020 04:30:00
69010     11/07/2020 13:10:00
19925     14/08/2018 14:00:00
201452    04/12/2019 05:00:00
34982     03/07/2019 19:30:00
Name: FECHA_ACCIDENTE, dtype: object

In [218]:
df["FECHA_ACCIDENTE"] = pd.to_datetime(df["FECHA_ACCIDENTE"], dayfirst=True)
df["FECHA_ACCIDENTE"].sample(20)

105840   2017-12-11 18:40:00
97738    2018-04-08 17:00:00
69519    2020-08-05 10:33:00
186105   2020-07-23 08:00:00
160452   2016-12-13 00:00:00
142781   2016-02-18 12:00:00
251429   2017-07-21 10:30:00
77648    2016-04-17 15:00:00
209259   2018-09-08 12:00:00
28563    2019-01-21 06:50:00
74228    2016-01-25 07:25:00
166641   2017-05-18 00:00:00
211699   2018-10-14 23:00:00
199596   2019-10-23 05:30:00
234253   2018-01-14 17:50:00
58425    2017-02-22 05:00:00
233499   2018-01-06 01:00:00
241231   2016-05-05 14:50:00
70886    2015-09-17 05:30:00
51886    2018-06-29 17:00:00
Name: FECHA_ACCIDENTE, dtype: datetime64[ns]

In [219]:
df["FECHA_ACCIDENTES"] = pd.to_datetime(df["FECHA_ACCIDENTES"])
df["FECHA_ACCIDENTES"].sample(20)

98382    2018-05-19 09:00:00+00:00
181296   2020-01-23 15:00:00+00:00
149630   2016-08-13 19:41:00+00:00
190307   2019-03-18 18:10:00+00:00
204452   2018-02-16 01:20:00+00:00
118086   2019-06-14 01:10:00+00:00
41019    2019-11-12 13:00:00+00:00
106728   2017-12-16 06:24:00+00:00
78560    2016-07-16 18:30:00+00:00
199823   2019-10-27 22:50:00+00:00
133051   2017-08-25 19:15:00+00:00
89305    2017-03-23 02:50:00+00:00
26309    2018-04-15 07:30:00+00:00
186862   2020-08-27 20:30:00+00:00
37231    2019-08-21 13:45:00+00:00
248347   2017-06-18 00:40:00+00:00
234193   2018-01-05 17:20:00+00:00
252936   2016-09-02 01:35:00+00:00
140094   2015-11-27 09:45:00+00:00
194352   2019-06-30 00:10:00+00:00
Name: FECHA_ACCIDENTES, dtype: datetime64[ns, UTC]

Dejamos ambas columnas de fecha con su tipo de datos correspondiente.

## GRAVEDAD_ACCIDENTE

In [220]:
df["GRAVEDAD_ACCIDENTE"].unique()

array(['Con heridos', 'Solo daños', 'Solo da\\xF1os', 'Con muertos'],
      dtype=object)

Cómo podemos ver tenemos registros con "Solo da\\xF1os", vamos a reemplazar esto con "Solo daños" y vamos a pasar todas las opciones a minuscula.

In [221]:
df["GRAVEDAD_ACCIDENTE"] = df["GRAVEDAD_ACCIDENTE"].map(lambda x : str(x).replace("Solo da\\xF1os", "Solo daños").lower())
df["GRAVEDAD_ACCIDENTE"] = df["GRAVEDAD_ACCIDENTE"].str.capitalize()
df["GRAVEDAD_ACCIDENTE"].unique()

array(['Con heridos', 'Solo daños', 'Con muertos'], dtype=object)

## MES

In [222]:
df["MES"].unique()

array([10, 11, 12,  9,  4,  3,  5,  6,  7,  8,  1,  2])

Podemos ver que los meses están correctamente, no tenemos valores extraños.

## NRO_RADICADO

In [223]:
df["NRO_RADICADO"].sample(20)

3837      1496201
15292     1614598
229060    1604647
21392     1639436
96364     1720800
238938    1473103
326       1509330
57487     1452520
188386    1658102
216931    1475959
54863     1460562
138298    1498258
239713    1530112
90340     1702542
167261    1531630
260933    1539071
35878     1679437
109616    1624562
242166    1552936
66721     1711529
Name: NRO_RADICADO, dtype: object

En este caso para NRO_RADICADO no se denota un patron que podamos utilizar para eliminar datos que no nos aporte de forma similar a CBML y EXPEDIENTE, así que no le realizaremos ninguna manipulación.

## NUMCOMUNA

In [224]:
df["NUMCOMUNA"].unique()

array(['10', '02', '01', '15', '14', '11', '07', '06', '04', '09', '05',
       '80', '13', '08', '16', '03', '12', '70', '60', '90', 'In', 'SN',
       '0', '50', 'AU', '5', '7', '9', '4', '3', '1', '2', '8', '6',
       'Sin Inf'], dtype=object)

Como vemos tenemos datos que no corresponden a comunas realmente, así que vamos a omitir los valores como:
- "AU"
- "Sin Inf"
- "In"
- "SN"
- "0"

Adicionalmente podemos ver valores como:
- 60
- 70
- 80
- 90

Estos valores los reemplazaremos por:
- 6
- 7
- 8
- 9

Respectivamente.

In [225]:
df = df[df["NUMCOMUNA"] != "AU"]
df = df[df["NUMCOMUNA"] != "Sin Inf"]
df = df[df["NUMCOMUNA"] != "In"]
df = df[df["NUMCOMUNA"] != "SN"]
df = df[df["NUMCOMUNA"] != "0"]

In [226]:
df["NUMCOMUNA"] = df["NUMCOMUNA"].map(lambda x : str(x).replace("60", "6").lower())
df["NUMCOMUNA"] = df["NUMCOMUNA"].map(lambda x : str(x).replace("70", "7").lower())
df["NUMCOMUNA"] = df["NUMCOMUNA"].map(lambda x : str(x).replace("80", "8").lower())
df["NUMCOMUNA"] = df["NUMCOMUNA"].map(lambda x : str(x).replace("90", "9").lower())
df["NUMCOMUNA"].unique()

array(['10', '02', '01', '15', '14', '11', '07', '06', '04', '09', '05',
       '8', '13', '08', '16', '03', '12', '7', '6', '9', '50', '5', '4',
       '3', '1', '2'], dtype=object)

Cómo vemos ya tenemos la información ajustada a las comunas de Medellín.

## BARRIO

In [227]:
df["BARRIO"].unique()

array(['Barrio Colón', 'San Diego', 'La Francia',
       'Santo Domingo Savio No. 1', 'Santa Fé', 'La Florida',
       'Los Conquistadores', 'La Alpujarra', 'Villa Nueva',
       'Facultad de Minas U. Nacional', 'La Esperanza',
       'Perpetuo Socorro', 'Campo Valdés No. 1', 'Miraflores',
       'Oleoducto', 'Terminal de Transporte', 'Gerona',
       'U.D. Atanasio Girardot', 'Alejandro Echavarría',
       'Cabecera San Antonio de Prado', 'Manila', 'Los Balsos No.2',
       'Moravia', 'Tenche', 'Universidad Nacional', 'Los Colores',
       'Caribe', 'Juan XXIII La Quiebra', 'Belalcázar', 'Santa Cruz',
       'Sucre', 'La Candelaria', 'El Estadio', 'Suramericana',
       'La Gloria', 'Centro Administrativo', 'Las Estancias',
       'Aures No. 2', 'Los Ángeles', 'Castilla', 'La Salle', 'La América',
       'Picacho', 'Bosques de San Pablo', 'La Aguacatala',
       'Playón de Los Comuneros', 'Estación Villa', 'La Colina',
       'Popular', 'Villa del Socorro', 'Cucaracho',
       'Santa 

Podemos ver que existen nombres de barrios donde al tildar algúna vocal generaron una secuencia de escape que representa un carácter codificado en UTF-8 en hexadecimal esto hace el nombre del barrio bastante ilegible, vamos a darle manejo a esto reemplazado el código respectivo a la vocal o letra especifica.

In [228]:
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).lower())
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xed","i").replace("í","i"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xf3","o").replace("ó","o"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xfa","u").replace("ú","u"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xf1","ñ"))

In [229]:
df["BARRIO"] = df["BARRIO"].str.capitalize()

In [230]:
df["BARRIO"].unique()

array(['Barrio colon', 'San diego', 'La francia',
       'Santo domingo savio no. 1', 'Santa fe', 'La florida',
       'Los conquistadores', 'La alpujarra', 'Villa nueva',
       'Facultad de minas u. nacional', 'La esperanza',
       'Perpetuo socorro', 'Campo valdes no. 1', 'Miraflores',
       'Oleoducto', 'Terminal de transporte', 'Gerona',
       'U.d. atanasio girardot', 'Alejandro echavarria',
       'Cabecera san antonio de prado', 'Manila', 'Los balsos no.2',
       'Moravia', 'Tenche', 'Universidad nacional', 'Los colores',
       'Caribe', 'Juan xxiii la quiebra', 'Belalcazar', 'Santa cruz',
       'Sucre', 'La candelaria', 'El estadio', 'Suramericana',
       'La gloria', 'Centro administrativo', 'Las estancias',
       'Aures no. 2', 'Los angeles', 'Castilla', 'La salle', 'La america',
       'Picacho', 'Bosques de san pablo', 'La aguacatala',
       'Playon de los comuneros', 'Estacion villa', 'La colina',
       'Popular', 'Villa del socorro', 'Cucaracho',
       'Santa 

## COMUNA

In [231]:
df["COMUNA"].unique()

array(['La Candelaria', 'Santa Cruz', 'Popular', 'Guayabal', 'El Poblado',
       'Laureles Estadio', 'Robledo', 'Doce de Octubre', 'Aranjuez',
       'Buenos Aires', 'Castilla',
       'Corregimiento de San Antonio de Prado', 'San Javier',
       'Villa Hermosa', 'Belén', 'Manrique', 'La América',
       'Corregimiento de Altavista', 'Corregimiento de San Cristóbal',
       'Corregimiento de Santa Elena', 'In',
       'Corregimiento de San Sebastián de Palmitas', 'Bel\\xE9n',
       'La Am\\xE9rica', 'Corregimiento de San Crist\\xF3bal',
       'Corregimiento de San Sebasti\\xE1n de Palmitas', 'AU'],
      dtype=object)

Para la columna "COMUNA" tenemos un caso similar al barrio, vamos a darle el mismo tratamiento.

In [232]:
df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).lower())
df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))

df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))

df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).replace("\\xf3","o").replace("ó","o"))

In [233]:
df = df[df["COMUNA"] != "au"]
df = df[df["COMUNA"] != "in"]
df = df[df["COMUNA"] != "nan"]
df = df[df["COMUNA"] != "sn"]
df = df[df["COMUNA"] != "no georef"]
df = df[df["COMUNA"] != "sin inf"]
df = df[df["COMUNA"] != "0"]

In [234]:
df["COMUNA"].unique()

array(['la candelaria', 'santa cruz', 'popular', 'guayabal', 'el poblado',
       'laureles estadio', 'robledo', 'doce de octubre', 'aranjuez',
       'buenos aires', 'castilla',
       'corregimiento de san antonio de prado', 'san javier',
       'villa hermosa', 'belen', 'manrique', 'la america',
       'corregimiento de altavista', 'corregimiento de san cristobal',
       'corregimiento de santa elena',
       'corregimiento de san sebastian de palmitas'], dtype=object)

In [235]:
df["COMUNA"] = df["COMUNA"].str.capitalize()

## LOCATION, X, Y

In [236]:
df["LOCATION"].head(10)

0    [-75.5688011014, 6.24312304123]
1    [-75.5693883283, 6.22870030622]
2    [-75.5503911403, 6.29869502502]
3    [-75.5442195069, 6.30019383306]
4     [-75.578180998, 6.21699243701]
5      [-75.561118009, 6.2047775959]
6    [-75.5810144962, 6.24576066323]
7    [-75.5752597216, 6.24685341912]
8    [-75.5624178534, 6.25341454089]
9    [-75.5931430393, 6.27330476643]
Name: LOCATION, dtype: object

In [237]:
df["X"].head(10)

0    834949.69
1    834880.17
2    837004.94
3    837688.53
4    833903.11
5    835788.30
6    833598.49
7    834235.89
8    835659.53
9    832264.64
Name: X, dtype: float64

In [238]:
df["Y"].head(10)

0    1182357.34
1    1180762.02
2    1188499.17
3    1188663.05
4    1179469.60
5    1178113.02
6    1182652.97
7    1182772.04
8    1183493.83
9    1185703.90
Name: Y, dtype: float64

Estas columnas aparentemente se encuentran bien, así que no los vamos a modificar.

## Manejo de columnas

Ahora vamos a eliminar algunas columnas que tienen datos redundantes con otras, por ejemplo:

- Direccion encasillada redunda con la columna Direccion
- Fecha accidentes redunda con la columna Fecha accidente
- Expediente redunda con la columna Nro radicado

In [239]:
df.drop(labels=["DIRECCION ENCASILLADA","EXPEDIENTE","FECHA_ACCIDENTES","NRO_RADICADO"],axis=1,inplace=True)

In [240]:
df

,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DISEÑO,FECHA_ACCIDENTE,GRAVEDAD_ACCIDENTE,MES,NUMCOMUNA,BARRIO,COMUNA,LOCATION,X,Y
0,2015,1013,Caida ocupante,CR 46 CL 43,Tramo de via,2015-10-21 05:58:00,Con heridos,10,10,Barrio colon,La candelaria,"[-75.5688011014, 6.24312304123]",834949.69,1182357.34
1,2015,1020,Choque,CR 43 A CL 29,Tramo de via,2015-11-05 03:00:00,Solo daños,11,10,San diego,La candelaria,"[-75.5693883283, 6.22870030622]",834880.17,1180762.02
2,2015,0205,Otro,CR 46 CL 37,Tramo de via,2015-10-21 07:40:00,Con heridos,10,02,La francia,Santa cruz,"[-75.5503911403, 6.29869502502]",837004.94,1188499.17
3,2015,0101,Atropello,CR 34 CL 112,Tramo de via,2015-10-21 12:00:00,Con heridos,10,01,Santo domingo savio no. 1,Popular,"[-75.5442195069, 6.30019383306]",837688.53,1188663.05
4,2015,1504,Choque,CR 50 CL 14,Tramo de via,2015-11-03 06:16:00,Solo daños,11,15,Santa fe,Guayabal,"[-75.578180998, 6.21699243701]",833903.11,1179469.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270760,2017,0913,Atropello,CR 28 CL 38 F,Lote o predio,2017-04-21 17:30:00,Con heridos,4,09,Loreto,Buenos aires,"[-75.5559460731, 6.23260034372]",836369.48,1181189.27
270761,2017,1115,Caida ocupante,CL 53 CR 69,Lote o predio,2017-03-10 16:30:00,Con heridos,3,11,Cuarta brigada,Laureles estadio,"[-75.5851522144, 6.26044928627]",833145.10,1184279.21
270762,2017,0513,Choque,CR 64 C CL 92,Tramo de via,2017-03-10 17:10:00,Solo daños,3,05,Francisco antonio zea,Castilla,"[-75.5713587099, 6.28457841509]",834679.61,1186944.12
270763,2017,0607,Caida ocupante,CR 78 A CL 92,Tramo de via,2017-03-10 20:00:00,Con heridos,3,06,Kennedy,Doce de octubre,"[-75.5831132045, 6.28957484792]",833380.07,1187500.58


## Archivo de salida

Ahora vamos generar el archivo con los datos tratados.

In [242]:
df.to_csv("dataTratada.csv", index=False)